In [1]:
import numpy as np, matplotlib.pyplot as plt, matplotlib as mpl
import pcmdpy_gpu as ppy
import pandas as pd
from tqdm import tqdm
from seaborn.apionly import color_palette
%matplotlib inline

/n/home01/bcook/.conda/envs/py36/lib/python3.6/site-packages/matplotlib/__init__.py:855: MatplotlibDeprecationWarning: 
examples.directory is deprecated; in the future, examples will be found relative to the 'datapath' directory.
  "found relative to the 'datapath' directory.".format(key))
/n/home01/bcook/.conda/envs/py36/lib/python3.6/site-packages/matplotlib/__init__.py:846: MatplotlibDeprecationWarning: 
The text.latex.unicode rcparam was deprecated in Matplotlib 2.2 and will be removed in 3.1.
  "2.2", name=key, obj_type="rcparam", addendum=addendum)
/n/home01/bcook/.conda/envs/py36/lib/python3.6/site-packages/seaborn/apionly.py:9: UserWarning: As seaborn no longer sets a default style on import, the seaborn.apionly module is deprecated. It will be removed in a future version.
  warnings.warn(msg, UserWarning)


In [2]:
plt.style.use('cook')

In [3]:
from setup_files.mocks_paper1.mock_models import models as mock_models, run_names as mock_names, results as mock_res

In [4]:
from setup_files.m31_paper2.m31_models import models as m31_models, run_names as m31_names, results as m31_res

/n/home01/bcook/.local/lib/python3.6/site-packages/numpy/lib/arraysetops.py:472: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)
/n/home01/bcook/pCMDs/pixcmd/pcmdpy/pcmdpy/plotting/results.py:83: RuntimeWarning: invalid value encountered in sqrt
  logzerrs.append(np.sqrt(logzvar))


In [5]:
models = mock_models.copy()
models.update(m31_models)

run_names = mock_names.copy()
run_names.update(m31_names)

results = mock_res.copy()
results.update(m31_res)

# Setup

In [6]:
f_mock = ppy.instrument.default_m31_filters()
f_summer = ppy.instrument.m31_summer_filters()
f_winter = ppy.instrument.m31_winter_filters()

In [7]:
iso_mock = ppy.isochrones.Isochrone_Model(f_mock, mag_system='vega')
iso_summer = ppy.isochrones.Isochrone_Model(f_summer, mag_system='ab')
iso_winter = ppy.isochrones.Isochrone_Model(f_winter, mag_system='ab')

In [8]:
driv_mock = ppy.driver.Driver(iso_mock)
driv_summer = ppy.driver.Driver(iso_summer)
driv_winter = ppy.driver.Driver(iso_winter)

Please wait while the GPU states are initializedPlease wait while the GPU states are initializedPlease wait while the GPU states are initialized

In [9]:
magbins = np.arange(10, 45, 0.05)
colorbins = np.arange(-1.5, 5.6, 0.05)  # fairly insensitive to distance
bins = [magbins, colorbins]

magbins_n = np.arange(10, 45, 0.02)
colorbins_n = np.arange(-1.5, 5.6, 0.02)  # fairly insensitive to distance
narrow_bins = [magbins_n, colorbins_n]

magbins_w = np.arange(10, 45, 0.1)
colorbins_w = np.arange(-1.5, 5.6, 0.1)  # fairly insensitive to distance
wide_bins = [magbins_w, colorbins_w]

In [10]:
logls = {}
logls_true = {}

In [ ]:
N_sample = 10
for k, r in tqdm(results.items()):
    season = getattr(r, 'season', '')
    if 'summer' in season:
        driv = driv_summer
    elif 'winter' in season:
        driv = driv_winter
    else:
        driv = driv_mock
    Nim_mock = 256
    Nim = 512
    if k == 'mock_10':
        Nim = 256
    if k == 'mock_11':
        Nim = 1024
    like_mode = 2
    if k == 'mock_9':
        like_mode = 3
    these_bins = bins
    if k in ['mock_29', 'mock_31']:
        these_bins = wide_bins
    elif k in ['mock_30', 'mock_32']:
        these_bins = narrow_bins
    logl = []
    logl_true = []
    pcmd_orig = driv.simulate(r.best_model, Nim_mock, fixed_seed=True)[0]
    if r.true_model is not None:
        pcmd_true = driv.simulate(r.true_model, Nim_mock, fixed_seed=True)[0]
    for _ in range(N_sample):
        pcmd_new = driv.simulate(r.best_model, Nim)[0]
        driv.initialize_data(pcmd_orig, bins=these_bins)
        logl.append(driv.loglike(pcmd_new, like_mode=like_mode))
        if r.true_model is not None:
            driv.initialize_data(pcmd_true, bins=these_bins)
            logl_true.append(driv.loglike(pcmd_new, like_mode=like_mode))
    logls[k] = logl
    logls_true[k] = logl_true
print(logls)
print(logls_true)

 95%|█████████▍| 107/113 [33:20<01:22, 13.79s/it]

In [ ]:
logl_means = {k: np.mean(v) for k,v in logls.items()}
logl_true_means = {k: np.mean(v) for k,v in logls_true.items()}

In [ ]:
df_logl = pd.DataFrame.from_dict(logl_means, 'index', columns=['logl_mean'])
df_true = pd.DataFrame.from_dict(logl_true_means, 'index', columns=['logl_true'])
df_logl = df_logl.merge(df_true, how='outer', left_index=True, right_index=True)
df_logl.head()

In [ ]:
df_mock = df_logl[~np.in1d(df_logl.index.values, list(m31_res.keys()))]
df_m31 = df_logl[np.in1d(df_logl.index.values, list(m31_res.keys()))]
df_m31.drop('logl_true', axis=1, inplace=True)

In [18]:
df_mock.to_csv('setup_files/mocks_paper1/max_logls.csv', float_format='%.3f', index_label='run_name')

In [17]:
df_m31.to_csv('setup_files/m31_paper2/max_logls.csv', float_format='%.3f', index_label='run_name')